In [3]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error,r2_score
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk 
import webbrowser
import os

In [4]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Khushal\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [6]:
apps_df=pd.read_csv(r"E:\NULLCLASS\datasets\Play Store Data.csv")
reviews_df=pd.read_csv(r"E:\NULLCLASS\datasets\User Reviews.csv")

In [7]:
apps_df.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [5]:
reviews_df.head()

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
2,10 Best Foods for You,NaN,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000


In [8]:
# Step 2 - DATA CLEANING
apps_df = apps_df.dropna(subset=['Rating'])
for column in apps_df.columns:
    apps_df[column].fillna(apps_df[column].mode()[0], inplace=True)
apps_df.drop_duplicates(inplace=True)
apps_df = apps_df[apps_df['Rating'] <= 5]

# Verify the column name in reviews_df
if 'translated_review' in reviews_df.columns:
    reviews_df.dropna(subset=['translated_review'], inplace=True)
else:
    print("Column 'translated_review' does not exist in reviews_df.")


Column 'translated_review' does not exist in reviews_df.


C:\Users\Khushal\AppData\Local\Temp\ipykernel_23132\3255238745.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  apps_df[column].fillna(apps_df[column].mode()[0], inplace=True)


In [9]:
apps_df.dtypes

App                object
Category           object
Rating            float64
Reviews            object
Size               object
Installs           object
Type               object
Price              object
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object
dtype: object

In [10]:
#convert the installs columns to numeric by removing commas and +
apps_df['Installs']=apps_df['Installs'].str.replace(',','').str.replace('+','').astype(int)

#convert price column to numeric after removing $
apps_df['Price']=apps_df['Price'].str.replace('$','').astype(float)

In [9]:
apps_df.dtypes

App                object
Category           object
Rating            float64
Reviews            object
Size               object
Installs            int32
Type               object
Price             float64
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object
dtype: object

In [11]:
merged_df=pd.merge(apps_df,reviews_df,on='App',how='inner')

In [11]:
merged_df.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,500000,Free,0.0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,A kid's excessive ads. The types ads allowed a...,Negative,-0.250,1.000000
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,500000,Free,0.0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,It bad >:(,Negative,-0.725,0.833333
2,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,500000,Free,0.0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,like,Neutral,0.000,0.000000
3,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,500000,Free,0.0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,NaN,NaN,NaN,NaN
4,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,500000,Free,0.0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,I love colors inspyering,Positive,0.500,0.600000


In [12]:
#Step 3 - Data Transmission
def convert_size(size):
    if 'M' in size:
        return float(size.replace('M',''))
    elif 'k' in size:
        return float(size.replace('k',''))/1024
    else:
        return np.nan
apps_df['Size']=apps_df['Size'].apply(convert_size)

In [13]:
apps_df

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19.0,10000,Free,0.0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14.0,500000,Free,0.0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7,5000000,Free,0.0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25.0,50000000,Free,0.0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8,100000,Free,0.0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10834,FR Calculator,FAMILY,4.0,7,2.6,500,Free,0.0,Everyone,Education,"June 18, 2017",1.0.0,4.1 and up
10836,Sya9a Maroc - FR,FAMILY,4.5,38,53.0,5000,Free,0.0,Everyone,Education,"July 25, 2017",1.48,4.1 and up
10837,Fr. Mike Schmitz Audio Teachings,FAMILY,5.0,4,3.6,100,Free,0.0,Everyone,Education,"July 6, 2018",1.0,4.1 and up
10839,The SCP Foundation DB fr nn5n,BOOKS_AND_REFERENCE,4.5,114,NaN,1000,Free,0.0,Mature 17+,Books & Reference,"January 19, 2015",Varies with device,Varies with device


In [14]:
#logarithmic 
apps_df['Log_installs']=np.log(apps_df['Installs'])

In [15]:
apps_df['Reviews']=apps_df['Reviews'].astype(int)

In [16]:
apps_df['Log_Reviews']=np.log(apps_df['Reviews'])

In [17]:
apps_df.dtypes

App                object
Category           object
Rating            float64
Reviews             int64
Size              float64
Installs            int64
Type               object
Price             float64
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object
Log_installs      float64
Log_Reviews       float64
dtype: object

In [18]:
def rating_group(rating):
    if rating >=4:
        return 'Top rated app'
    elif rating >=3:
        return 'Above avg'
    elif rating >=2:
        return 'Avg'
    else:
        return 'Below avg'
apps_df['Rating_Group']=apps_df['Rating'].apply(rating_group)    

In [19]:
#Revenue column
apps_df['Revenue']=apps_df['Price']*apps_df['Installs']

In [20]:
# Step 4 - Sentiment Analysis

sia = SentimentIntensityAnalyzer()

In [21]:
#Polarity scores in SIA
#Positive,negative,neutral & compound 

In [22]:
review="this app is amazing! I love the new features"
sentiment_score=sia.polarity_scores(review)
print(sentiment_score)

{'neg': 0.0, 'neu': 0.42, 'pos': 0.58, 'compound': 0.8516}


In [23]:
review="this app is very bad! I hate the new features"
sentiment_score=sia.polarity_scores(review)
print(sentiment_score)

{'neg': 0.535, 'neu': 0.465, 'pos': 0.0, 'compound': -0.8427}


In [24]:
review="this app is okay"
sentiment_score=sia.polarity_scores(review)
print(sentiment_score)

{'neg': 0.0, 'neu': 0.612, 'pos': 0.388, 'compound': 0.2263}


In [25]:
reviews_df['Sentiment_Score']=reviews_df['Translated_Review'].apply(lambda x : sia.polarity_scores(str(x))['compound'])

In [26]:
reviews_df

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity,Sentiment_Score
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333,0.9531
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462,0.6597
2,10 Best Foods for You,NaN,NaN,NaN,NaN,0.0000
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000,0.6249
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000,0.6369
...,...,...,...,...,...,...
64290,Houzz Interior Design Ideas,NaN,NaN,NaN,NaN,0.0000
64291,Houzz Interior Design Ideas,NaN,NaN,NaN,NaN,0.0000
64292,Houzz Interior Design Ideas,NaN,NaN,NaN,NaN,0.0000
64293,Houzz Interior Design Ideas,NaN,NaN,NaN,NaN,0.0000


In [27]:
apps_df['Last Updated']=pd.to_datetime(apps_df['Last Updated'],errors='coerce')

In [28]:
apps_df['Year']=apps_df['Last Updated'].dt.year

In [29]:
apps_df.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Log_installs,Log_Reviews,Rating_Group,Revenue,Year
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19.0,10000,Free,0.0,Everyone,Art & Design,2018-01-07,1.0.0,4.0.3 and up,9.210340,5.068904,Top rated app,0.0,2018
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14.0,500000,Free,0.0,Everyone,Art & Design;Pretend Play,2018-01-15,2.0.0,4.0.3 and up,13.122363,6.874198,Above avg,0.0,2018
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7,5000000,Free,0.0,Everyone,Art & Design,2018-08-01,1.2.4,4.0.3 and up,15.424948,11.379508,Top rated app,0.0,2018
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25.0,50000000,Free,0.0,Teen,Art & Design,2018-06-08,Varies with device,4.2 and up,17.727534,12.281384,Top rated app,0.0,2018
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8,100000,Free,0.0,Everyone,Art & Design;Creativity,2018-06-20,1.1,4.4 and up,11.512925,6.874198,Top rated app,0.0,2018


In [38]:
import plotly.express as px
fig=px.bar(x=["A","B","C"],y=[1,3,2],title="sample bar chart")
fig.show()

In [39]:
# Step 5 - Plotly part 1

fig.write_html("Interactive_plot.html")
#static visualizations
#Interactive visualizations

In [40]:
html_files_path='./'
if not os.path.exists(html_files_path):
    os.makedirs(html_files_path)

In [41]:
plot_containers=""

In [42]:
#save each plotly figure to an html file

def save_plot_as_html(fig,filename,insight):
    global plot_containers
    filepath=os.path.join(html_files_path,filename)
    html_content=pio.to_html(fig,full_html=False,include_plotlyjs='inline')

    #append the plot and its insight to plot_containers

    plot_containers += f"""
     <div class="plot_containers" id="{filename}" onclick="openPlot('{filename}')">
     <div class="plot">{html_content}</div>
     <div class="insights">{insight}</div>
     </div>
     """
    
    fig.write_html(filepath,full_html=False,include_plotlyjs='inline')

In [43]:
plot_width=400
plot_height=300
plot_bg_color="black"
text_color="white"
title_font={'size:16'}
axis_font={'size:12'}

In [44]:
category_count = apps_df['Category'].value_counts().nlargest(10)

fig1 = px.bar(
    x=category_count.index,
    y=category_count.values,
    labels={'x': 'Category', 'y': 'Count'},
    title="Top Categories on Play Store",
    color=category_count.index,
    color_discrete_sequence=px.colors.sequential.Plasma,
    width=400,
    height=300
)

fig1.update_layout(
    plot_bgcolor="black",
    paper_bgcolor="black",
    font_color="white",
    title_font={'size': 16},
    xaxis=dict(title_font={'size': 12}),
    yaxis=dict(title_font={'size': 12}),
    margin=dict(l=10, r=10, t=30, b=10)
)

# Uncomment the next line if you want to add trace-specific styling
# fig1.update_traces(marker=dict(pattern=dict(line=dict(color='white', width=1))))

# Define the save function if not already defined
def save_plot_as_html(figure, filename, description):
    figure.write_html(filename)
    print(description)

save_plot_as_html(fig1, "Category Graph 1.html", "The top categories on the play store are dominated by tools, entertainment, and productivity apps.")


The top categories on the play store are dominated by tools, entertainment, and productivity apps.


In [45]:
# Step 6 - Plotly part 2
#fig 2

type_counts=apps_df['Type'].value_counts()
fig2 = px.pie(
    values=type_counts.values,
    names=type_counts.index,
    title="App type distribution",
    color_discrete_sequence=px.colors.sequential.RdBu,
    width=400,
    height=300
)

fig2.update_layout(
    plot_bgcolor="black",
    paper_bgcolor="black",
    font_color="white",
    title_font={'size': 16},
    margin=dict(l=10, r=10, t=30, b=10)
)

save_plot_as_html(fig2, "Category Graph 2.html", "Most apps on the playstore are free,indicating a strategy to attract users first and monetize through ads and or in app purchases")


Most apps on the playstore are free,indicating a strategy to attract users first and monetize through ads and or in app purchases


In [46]:
#fig 3

fig3= px.histogram(
    apps_df,
    x='Rating',
    nbins=20,
    title="Rating Distribution",
    color_discrete_sequence=['#636EFA'],
    width=400,
    height=300
)

fig2.update_layout(
    plot_bgcolor="black",
    paper_bgcolor="black",
    font_color="white",
    title_font={'size': 16},
    xaxis=dict(title_font={'size': 12}),
    yaxis=dict(title_font={'size': 12}),
    margin=dict(l=10, r=10, t=30, b=10)
)

save_plot_as_html(fig3, "Rating Graph 3.html", "Ratings are skewed towards higher values,suggesting that most apps are rated favorably by users")


Ratings are skewed towards higher values,suggesting that most apps are rated favorably by users


In [47]:
#fig 4

sentiment_counts=reviews_df['Sentiment_Score'].value_counts()
fig4= px.bar(
    x=sentiment_counts.index,
    y=sentiment_counts.index,
    labels={'x':'Sentiment_Score','y':'Count'},
    title="Sentiment Distribution",
    color=sentiment_counts.index,
    color_discrete_sequence=px.colors.sequential.RdPu,
    width=400,
    height=300
)

fig4.update_layout(
    plot_bgcolor="black",
    paper_bgcolor="black",
    font_color="white",
    title_font={'size': 16},
    xaxis=dict(title_font={'size': 12}),
    yaxis=dict(title_font={'size': 12}),
    margin=dict(l=10, r=10, t=30, b=10)
)

save_plot_as_html(fig4, "Rating Graph 4.html", "Sentiment in reviews show a mix of positive and negative feedback,with a slight lean towards positive sentiments")


Sentiment in reviews show a mix of positive and negative feedback,with a slight lean towards positive sentiments


In [48]:
#fig 5
installS_by_category = apps_df.groupby('Category')['Installs'].sum().nlargest(10)

fig5 = px.bar(
    x=installS_by_category.index,
    y=installS_by_category.values,
    labels={'x': 'Category', 'y': 'Installs'},
    title="Installs by Category",
    color=installS_by_category.index,  # Ensures color matches the 10 categories
    color_discrete_sequence=px.colors.sequential.Blues,
    width=400,
    height=300
)

fig5.update_layout(
    plot_bgcolor="black",
    paper_bgcolor="black",
    font_color="white",
    title_font={'size': 16},
    xaxis=dict(title_font={'size': 12}),
    yaxis=dict(title_font={'size': 12}),
    margin=dict(l=10, r=10, t=30, b=10)
)

# Assuming save_plot_as_html is a custom function to save the plot
save_plot_as_html(
    fig5,
    "Rating Graph 5.html",
    "The Categories with the most installs are social and communication apps, reflecting their broad appeal and daily usage."
)



The Categories with the most installs are social and communication apps, reflecting their broad appeal and daily usage.


In [49]:
# step - 
#fig 6

import plotly.express as px

# Define the colors and fonts
plot_bg_color = "white"
text_color = "black"
title_font = {"size": 20, "color": "black"}
axis_font = {"size": 14, "color": "black"}

# Updates per year
updates_per_year = apps_df['Last Updated'].dt.year.value_counts().sort_index()
fig6 = px.line(
    x=updates_per_year.index,
    y=updates_per_year.values,
    labels={'x': 'Year', 'y': 'No of updates'},
    title="No of updates over the years",
    color_discrete_sequence=['#AB63FA'],
    width=plot_width,
    height=plot_height
)

# Update layout
fig6.update_layout(
    plot_bgcolor=plot_bg_color,
    paper_bgcolor=plot_bg_color,
    font_color=text_color,
    title_font=title_font,
    xaxis=dict(title_font=axis_font),
    yaxis=dict(title_font=axis_font),
    margin=dict(l=10, r=10, t=30, b=10)
)

# Save the plot as HTML
save_plot_as_html(
    fig6,
    "Update Graph 6.html",
    "Updates have been increased over the years, showing that developers are actively maintaining and improving their apps"
)


Updates have been increased over the years, showing that developers are actively maintaining and improving their apps


In [50]:
#fig 7 

revenue_by_category=apps_df.groupby('Category')['Revenue'].sum().nlargest(10)
fig7 = px.bar(
    x=revenue_by_category.index,
    y=revenue_by_category.values,
    labels={'x': 'Category', 'y': 'Revenue'},
    title="Revenue  by Category",
    color=installS_by_category.index,  # Ensures color matches the 10 categories
    color_discrete_sequence=px.colors.sequential.Greens,
    width=400,
    height=300
)

fig7.update_layout(
    plot_bgcolor="black",
    paper_bgcolor="black",
    font_color="white",
    title_font={'size': 16},
    xaxis=dict(title_font={'size': 12}),
    yaxis=dict(title_font={'size': 12}),
    margin=dict(l=10, r=10, t=30, b=10)
)

# Assuming save_plot_as_html is a custom function to save the plot
save_plot_as_html(
    fig7,
    "Revenue Graph 7.html","Categories such as Business and productivity lead in revenue generation , indicating their monetization potential")



Categories such as Business and productivity lead in revenue generation , indicating their monetization potential


In [51]:
#fig 8 

genre_counts=apps_df['Genres'].str.split(';',expand=True).stack().value_counts().nlargest(10)
fig8 = px.bar(
    x=genre_counts.index,
    y=genre_counts.values,
    labels={'x': 'Genre', 'y': 'count'},
    title="Top Genres",
    color=installS_by_category.index,  # Ensures color matches the 10 categories
    color_discrete_sequence=px.colors.sequential.OrRd,
    width=400,
    height=300
)

fig8.update_layout(
    plot_bgcolor="black",
    paper_bgcolor="black",
    font_color="white",
    title_font={'size': 16},
    xaxis=dict(title_font={'size': 12}),
    yaxis=dict(title_font={'size': 12}),
    margin=dict(l=10, r=10, t=30, b=10)
)

save_plot_as_html(
    fig8,
    "Genre Graph 8.html","Actions and Casual genres are the most common ,reflecting users' preference for engaging and easy-to-play games")


Actions and Casual genres are the most common ,reflecting users' preference for engaging and easy-to-play games


In [52]:
#fig 9

fig9 = px.scatter(
    apps_df,
    x='Last Updated',
    y='Rating',
    color='Type',
    title="Impact of last update on rating",
    color_discrete_sequence=px.colors.qualitative.Vivid,
    width=400,
    height=300
)

fig9.update_layout(
    plot_bgcolor="black",
    paper_bgcolor="black",
    font_color="white",
    title_font={'size': 16},
    xaxis=dict(title_font={'size': 12}),
    yaxis=dict(title_font={'size': 12}),
    margin=dict(l=10, r=10, t=30, b=10)
)

save_plot_as_html(
    fig9,
    "Update Graph 9.html","The scatter plot shows a weak correlation between the last update and ratings,suggesting that more frequent updates don't alwyas result in better ratings ")

The scatter plot shows a weak correlation between the last update and ratings,suggesting that more frequent updates don't alwyas result in better ratings 


In [53]:
#fig 10

fig10 = px.box(
    apps_df,
    x='Type',
    y='Rating',
    color='Type',
    title="Rating for paid vs Free apps",
    color_discrete_sequence=px.colors.qualitative.Pastel,
    width=400,
    height=300
)

fig10.update_layout(
    plot_bgcolor="black",
    paper_bgcolor="black",
    font_color="white",
    title_font={'size': 16},
    xaxis=dict(title_font={'size': 12}),
    yaxis=dict(title_font={'size': 12}),
    margin=dict(l=10, r=10, t=30, b=10)
)

save_plot_as_html(
    fig10,
    "Paid free Graph 10.html","Paid apps generally have higher ratings comapred to free apps,suggesting that users expect higher quality from apps they pay for")


Paid apps generally have higher ratings comapred to free apps,suggesting that users expect higher quality from apps they pay for


In [54]:
    plot_containers_split=plot_containers.split('</div')

In [55]:
if len(plot_containers_split) > 1:
    final_plot=plot_containers_split[-2]+'</div>'
else:
    final_plot=plot_containers    

In [56]:
import os
import webbrowser

# Path to the folder containing your plot HTML files
plot_folder_path = r"C:\Users\Khushal\Downloads\NULL_CLASS"  # Replace with your folder path

# List of all HTML files in the folder
plot_files = [file for file in os.listdir(plot_folder_path) if file.endswith(".html")]

# HTML template for the dashboard
dashboard_html = """
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>Google Play Store Review Analytics</title>
  <style>
    body {{
      font-family: Arial, sans-serif;
      background-color: #333;
      color: #fff;
      margin: 0;
      padding: 0;
    }}
    .header {{
      display: flex;
      align-items: center;
      justify-content: space-between;
      padding: 20px;
      background-color: #444;
    }}
    .header img {{
      height: 50px;
    }}
    .title {{
      text-align: center;
      flex-grow: 1;
      font-size: 24px;
      font-weight: bold;
    }}
    .container {{
      display: flex;
      flex-wrap: wrap;
      justify-content: center;
      padding: 20px;
    }}
    .plot-container {{
      margin: 20px;
      border: 2px solid #555;
      padding: 10px;
      background-color: #222;
      width: 90%; /* Uniform width for all plots */
      height: 500px; /* Uniform height for all plots */
    }}
    iframe {{
      width: 100%;
      height: 100%;
      border: none;
    }}
  </style>
</head>
<body>
  <div class="header">
    <img src="https://storage.googleapis.com/gd-prod/images/a910d418-7123-4bc4-aa3b-ef7e25e74ae6.60c498c559810aa0.webp" alt="Google logo">
    <div class="title">Google Play Store Review Analytics</div>
    <img src="https://www.logo.wine/a/logo/Google_Play/Google_Play-Icon-Logo.wine.svg" alt="Play Store logo">
  </div>
  <div class="container">
    {plots}
  </div>
</body>
</html>
"""

# Embed each plot into an iframe
combined_plots = ""
for plot_file in plot_files:
    plot_path = os.path.join(plot_folder_path, plot_file)
    iframe_code = f"<div class='plot-container'><iframe src='{plot_path}'></iframe></div>\n"
    combined_plots += iframe_code

# Create the final dashboard HTML
final_html = dashboard_html.format(plots=combined_plots)

# Save the combined HTML file
output_file = os.path.join(plot_folder_path, "combined_dashboard.html")
with open(output_file, "w", encoding="utf-8") as f:
    f.write(final_html)

# Open the combined dashboard in the default web browser
webbrowser.open('file://' + os.path.realpath(output_file))


True